In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip -q install ultralytics opencv-python numpy
from ultralytics import YOLO
import cv2, numpy as np, math, os, time, gradio as gr

# Cek GPU
import torch, platform
print("PyTorch:", torch.__version__, "| CUDA:", torch.cuda.is_available(), "| Python:", platform.python_version())


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 30.2 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
PyTorch: 2.8.0+cu126 | CUDA: True | Python: 3.12.12


In [4]:
from google.colab import files, drive
uploaded = files.upload()  # pilih file .mp4/.mov/.avi
in_path = list(uploaded.keys())[0]
print("Video input:", in_path)

Saving tes1.mp4 to tes1.mp4
Video input: tes1.mp4


In [5]:
model = YOLO('yolov8n-pose.pt')

In [6]:
# 0 nose, 1 l_eye, 2 r_eye, 3 l_ear, 4 r_ear,
# 5 l_shoulder, 6 r_shoulder, 7 l_elbow, 8 r_elbow,
# 9 l_wrist, 10 r_wrist, 11 l_hip, 12 r_hip,
# 13 l_knee, 14 r_knee, 15 l_ankle, 16 r_ankle

def angle(a, b, c):
    """
    Hitung sudut (derajat) di titik b, dari tiga titik (a-b-c).
    a, b, c: (x,y)
    """
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba = a - b
    bc = c - b
    cosang = np.dot(ba, bc) / (np.linalg.norm(ba)*np.linalg.norm(bc) + 1e-6)
    cosang = np.clip(cosang, -1.0, 1.0)
    return np.degrees(np.arccos(cosang))

def euclid(p1, p2):
    p1, p2 = np.array(p1), np.array(p2)
    return float(np.linalg.norm(p1 - p2))

def get_kpt_xy(kpts, idx):
    # kpts: shape (17,3) → (x, y, conf)
    return (float(kpts[idx,0]), float(kpts[idx,1]))

def is_visible(kpts, idx, thr=0.25):
    return float(kpts[idx,2]) >= thr

In [9]:
class RepetitionCounter:
    def __init__(self):
        self.pushup_cnt = 0; self.pushup_state = "up"       # start di "up"
        self.squat_cnt = 0;  self.squat_state  = "top"      # start di "top"
        self.jj_cnt = 0;     self.jj_state     = "closed"   # start di "closed"

    def update_pushup(self, k):
        # pakai siku (7,8), bahu (5,6), pergelangan (9,10), plus plank check bahu–pinggul–mata kaki
        if not all(is_visible(k, i) for i in [5,6,7,8,9,10,11,12,15,16]): return
        # sisi kiri & kanan → ambil rata-rata sudut siku
        L = angle(get_kpt_xy(k,5), get_kpt_xy(k,7), get_kpt_xy(k,9))
        R = angle(get_kpt_xy(k,6), get_kpt_xy(k,8), get_kpt_xy(k,10))
        elbow = (L+R)/2

        # cek “plank” (bahu-hip-ankle harus relatif lurus → sudut pinggul sekitar 160–200)
        hipL = angle(get_kpt_xy(k,5), get_kpt_xy(k,11), get_kpt_xy(k,15))
        hipR = angle(get_kpt_xy(k,6), get_kpt_xy(k,12), get_kpt_xy(k,16))
        plank = (hipL > 155 and hipR > 155)

        down_thr, up_thr = 90, 150
        if plank:
            if self.pushup_state == "up" and elbow < down_thr:
                self.pushup_state = "down"
            elif self.pushup_state == "down" and elbow > up_thr:
                self.pushup_cnt += 1
                self.pushup_state = "up"

    def update_squat(self, k):
        # pakai lutut (13,14), pinggul (11,12), mata kaki (15,16)
        if not all(is_visible(k, i) for i in [11,12,13,14,15,16]): return
        L = angle(get_kpt_xy(k,11), get_kpt_xy(k,13), get_kpt_xy(k,15))
        R = angle(get_kpt_xy(k,12), get_kpt_xy(k,14), get_kpt_xy(k,16))
        knee = (L+R)/2

        # depth check: pinggul lebih rendah dari lutut (koordinat y lebih besar → lebih bawah di gambar)
        hip_y = (get_kpt_xy(k,11)[1] + get_kpt_xy(k,12)[1]) / 2
        knee_y = (get_kpt_xy(k,13)[1] + get_kpt_xy(k,14)[1]) / 2
        depth_ok = hip_y > knee_y - 5  # toleransi kecil

        bottom_thr, top_thr = 90, 160
        if self.squat_state == "top" and knee < bottom_thr and depth_ok:
            self.squat_state = "bottom"
        elif self.squat_state == "bottom" and knee > top_thr:
            self.squat_cnt += 1
            self.squat_state = "top"

    def update_jj(self, k, frame_w):
        # kriteria open: tangan di atas kepala & kaki lebar
        if not all(is_visible(k, i) for i in [5,6,9,10,15,16,0]): return
        wristY = (get_kpt_xy(k,9)[1] + get_kpt_xy(k,10)[1]) / 2
        headY  = get_kpt_xy(k,0)[1]
        hands_up = wristY < headY  # di atas kepala

        ankles = euclid(get_kpt_xy(k,15), get_kpt_xy(k,16))
        feet_apart = ankles > 0.25*frame_w  # ambang relatif lebar frame

        if self.jj_state == "closed" and hands_up and feet_apart:
            self.jj_state = "open"
        elif self.jj_state == "open" and (not hands_up or not feet_apart):
            self.jj_cnt += 1
            self.jj_state = "closed"


In [11]:
def draw_text(img, text, x, y, scale=0.9, thickness=2):
    cv2.putText(img, text, (x,y), cv2.FONT_HERSHEY_SIMPLEX, scale, (0,0,0), thickness+3, cv2.LINE_AA)
    cv2.putText(img, text, (x,y), cv2.FONT_HERSHEY_SIMPLEX, scale, (255,255,255), thickness, cv2.LINE_AA)

def process_video(in_path, out_path="output.mp4", img_size=640):
    cap = cv2.VideoCapture(in_path)
    assert cap.isOpened(), "Error: Could not open video file."
    fps = cap.get(cv2.CAP_PROP_FPS) or 25
    W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    writer = cv2.VideoWriter(out_path, fourcc, fps, (W, H))

    counter = RepetitionCounter()

    while True:
        ok, frame = cap.read()
        if not ok: break

        # Inference YOLO (opsi: conf=0.25)
        res = model.predict(source=frame, imgsz=img_size, conf=0.25, verbose=False)[0]

        if len(res.keypoints) > 0:
            # pilih deteksi dengan skor tertinggi
            scores = res.boxes.conf.cpu().numpy() if res.boxes is not None else np.array([1.0]*len(res.keypoints))
            best_i = int(np.argmax(scores))
            kpts = res.keypoints[best_i].data[0].cpu().numpy().reshape(-1,3)

            def is_standing(kpts):
                head_y = get_kpt_xy(kpts, 0)[1]
                ankle_y = (get_kpt_xy(kpts,15)[1] + get_kpt_xy(kpts,16)[1]) / 2
                return (ankle_y - head_y) > 0.5 * ankle_y

            if is_standing(kpts):
                counter.update_jj(kpts, frame_w=W)
            else:
                counter.update_pushup(kpts)
                counter.update_squat(kpts)

            # render pose di frame
            plotted = res.plot()
            frame = plotted

        # overlay teks
        draw_text(frame, f"Push-up: {counter.pushup_cnt}", 20, 40)
        draw_text(frame, f"Squat  : {counter.squat_cnt}", 20, 80)
        draw_text(frame, f"J. Jack: {counter.jj_cnt}",   20, 120)

        writer.write(frame)

    cap.release(); writer.release()
    print("Selesai:", out_path)
    return out_path

out_path = process_video(in_path, out_path="output.mp4", img_size=640)

Selesai: output.mp4


In [12]:
from google.colab import files
files.download('output.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>